In [620]:
import glob
import pandas as pd
import time
import datetime as dt
import os
import numpy as np
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pathlib
from pathlib import Path
import pickle
import glob
from datetime import datetime
warnings.filterwarnings("ignore")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
import math
import plotly.express as px
import sys
from scipy.stats import skew, kurtosis
from scipy.stats import spearmanr
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.metrics import mean_tweedie_deviance
import sys
import umap as umap
def mean_absolute_percentage_error(y_true, y_predicted):
    y_true, y_predicted = np.array(y_true), np.array(y_predicted)
    return np.mean(np.abs((y_true - y_predicted) / y_true)) * 100
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
os.chdir('D:/BGM_Codebase/+200mesh_test\/200_mesh_final/Raw_Data')

In [621]:
cols=['Timestamp_From','Timestamp_To','Special_Filling_temp','Special_Filling_Flow','First_Agglo_Tank_Temp_T42','Tank_47_Temp',
 'PPT-48_Temperature','Fine_Seed_Tonnage','Fine_Seed_Flow','Fine_Seed_Density','Fine_Seed_Charge','Fine_Seed_SSA','Fine_Seed_3_5u',
 'Fine_Seed_45u','Fine_Seed_D50','PHE_Inlet_Temp','PHE_Outlet_Temp','Regular_Filling_Flow','Tank__52_Temp','Tank__56_Temp',
 'Coarse_Seed_1_Tonnage','Coarse_Seed_2_Tonnage','Coarse_Seed_2_Flow','Coarse_Seed_1_Density','Coarse_Seed_2_Density',
 'HAT_3_5u','HAT_45u','HAT_D50','Filling_Concentration','Filling_Ratio','Batch_Circulation_Hrs','Feed_Hydrate_D50',
 'Feed_Hydrate_SSA','Feed_Hydrate_100','Feed_Hydrate_325','Date']

final_cols=['Timestamp_To','Special_Filling_Flow','Fine_Seed_Tonnage','Fine_Seed_Flow','Fine_Seed_Density',
                        'Fine_Seed_SSA','Fine_Seed_3_5u','Fine_Seed_45u','Fine_Seed_D50','PHE_Inlet_Temp','Regular_Filling_Flow',
                        'Coarse_Seed_1_Tonnage','Coarse_Seed_2_Flow','HAT_3_5u','HAT_45u','HAT_D50','Filling_Ratio','Feed_Hydrate_D50','Feed_Hydrate_200','Feed_Hydrate_325','Feed_Hydrate_100']

columns_to_aggregate = ['Special_Filling_temp','Special_Filling_Flow','First_Agglo_Tank_Temp_T42',
                        'Fine_Seed_Tonnage','Fine_Seed_Flow','Fine_Seed_Density',
                       'PHE_Inlet_Temp','PHE_Outlet_Temp','Regular_Filling_Flow','Tank__52_Temp','Tank__56_Temp',
                        'Coarse_Seed_1_Tonnage','Coarse_Seed_2_Tonnage','Coarse_Seed_2_Flow','Coarse_Seed_1_Density',
                        'Coarse_Seed_2_Density',]
columns_to_resample = ['Fine_Seed_SSA','Fine_Seed_3_5u','Fine_Seed_45u','Fine_Seed_D50','HAT_3_5u','HAT_45u','HAT_D50',
                       'Filling_Concentration','Filling_Ratio','Feed_Hydrate_D50','Feed_Hydrate_SSA','Feed_Hydrate_200','Feed_Hydrate_325','Feed_Hydrate_100']

resample_frequency = '24H'

agg_functions = {
    'min': 'min',
    'max': 'max',
    'median': 'median',
    'mean': 'mean',
    'std': 'std', 
    'var': 'var', 
    'var_coeff': lambda x: x.std() / x.mean() if x.mean() != 0 else 0 
}

min_days = 39
max_days = 83
shifts_per_day = 1

def replace_outliers_with_median(data):
    # Convert relevant columns to numeric before calculations
    numeric_cols = data.select_dtypes(include=np.number).columns
    # Only process numeric columns for outlier replacement
    data_numeric = data[numeric_cols].copy()
    data_numeric[numeric_cols] = data_numeric[numeric_cols].apply(pd.to_numeric, errors='coerce')  # Convert to numeric, handle errors

    Q1 = data_numeric.quantile(0.25)
    Q3 = data_numeric.quantile(0.75)
    IQR = Q3 - Q1
    median = data_numeric.median()
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data_numeric = data_numeric.apply(lambda x: x.where((x.between(lower_bound[x.name], upper_bound[x.name])) | x.isna(), median[x.name]))

    # Update original data with replaced values
    data[numeric_cols] = data_numeric[numeric_cols]

    return data


def initial_data_preprocessing(treated_agg_data):
    # Convert Timestamp_To to datetime format
    treated_agg_data['Timestamp_To'] = pd.to_datetime(treated_agg_data['Timestamp_To'], errors='coerce')

    min_timestamp = treated_agg_data['Timestamp_To'].min()
    print(min_timestamp)

    if pd.isna(min_timestamp):  # Check if conversion failed
        raise ValueError("Timestamp_To contains invalid datetime values.")

    timestamp_1hr_before = min_timestamp - pd.Timedelta(hours=1)
    timestamp_2hr_before = min_timestamp - pd.Timedelta(hours=2)

    new_rows_dynamic = [
        {col: timestamp_1hr_before if col == 'Timestamp_To' else pd.NA for col in treated_agg_data.columns},
        {col: timestamp_2hr_before if col == 'Timestamp_To' else pd.NA for col in treated_agg_data.columns}
    ]
    
    new_rows_df_dynamic = pd.DataFrame(new_rows_dynamic)
    data_extended_dynamic = pd.concat([new_rows_df_dynamic, treated_agg_data], ignore_index=True)

    data_extended_dynamic.sort_values(by='Timestamp_To', inplace=True)
    data_extended_dynamic.reset_index(drop=True, inplace=True)

    data_extended_dynamic['Timestamp_To'] = data_extended_dynamic['Timestamp_To'] + pd.Timedelta(hours=1)
    data_extended_dynamic.set_index('Timestamp_To', inplace=True)

    return data_extended_dynamic


def data_aggregation(df):
    resampled_df = pd.DataFrame()
    for feature in df.columns:
        if feature in columns_to_aggregate:
            for agg_name, agg_func in agg_functions.items():
                new_feature_name = f'{feature}_{agg_name}'
                resampled_df[new_feature_name] = df[feature].resample(resample_frequency).apply(agg_func)
        elif feature in columns_to_resample:
            resampled_df[feature] = df[feature].resample(resample_frequency).mean()
            
    resampled_df.reset_index(inplace=True)
    resampled_df['Timestamp_To'] = resampled_df['Timestamp_To'] + pd.Timedelta(hours=6)
    return resampled_df

def data_filter(resampled_df):
    resampled_df_sorted = resampled_df.sort_values(by='Timestamp_To', ascending=False)
    
    for index, row in resampled_df_sorted.head(5).iterrows():
        if any(pd.isna(row[col]) for col in ['Feed_Hydrate_100', 'Feed_Hydrate_200', 'Feed_Hydrate_325']):
            resampled_df_sorted = resampled_df_sorted.drop(index)

    resampled_df = resampled_df_sorted.sort_values(by='Timestamp_To')
    return resampled_df


def calculate_lag_dates(target_date_str, min_days, max_days, shifts_per_day):
    target_date = datetime.strptime(target_date_str, "%Y-%m-%d %H:%M")
    min_lag_days = min_days // shifts_per_day
    max_lag_days = max_days // shifts_per_day
    max_lag_extra_shifts = max_days % shifts_per_day
    start_date_min_lag = target_date - timedelta(days=min_lag_days)
    start_date_max_lag = target_date - timedelta(days=max_lag_days, hours=max_lag_extra_shifts)
    return start_date_min_lag, start_date_max_lag

def preparing_lag_data(df, target_columns=['Feed_Hydrate_100', 'Feed_Hydrate_200', 'Feed_Hydrate_325']):
    for column in target_columns:
        for i in range(min_days, max_days + 1):
            nth_last_value = df[column].iloc[-(i - min_days + 1)]  # dynamic offset
            new_column_name = f'{column}_shift_{i}'
            df[new_column_name] = pd.NA
            df.at[df.index[-1], new_column_name] = nth_last_value
    
    df_lag_cleaned = df.dropna()
    return df_lag_cleaned


In [622]:
Agg_data=pd.read_csv("D:/BGM_Codebase/+200mesh_test/200_mesh_final/Raw_Data/Precip_Raw_Data_200.csv")

In [623]:
Agg_data.shape

(9588, 37)

In [624]:
Agg_data.head()

,Timestamp_From,Timestamp_To,Special_Filling_temp,Special_Filling_Flow,First_Agglo_Tank_Temp_T42,Tank_47_Temp,PPT-48_Temperature,Fine_Seed_Tonnage,Fine_Seed_Flow,Fine_Seed_Density,Fine_Seed_Charge,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp,PHE_Outlet_Temp,Regular_Filling_Flow,Tank__52_Temp,Tank__56_Temp,Coarse_Seed_1_Tonnage,Coarse_Seed_2_Tonnage,Coarse_Seed_2_Flow,Coarse_Seed_1_Density,Coarse_Seed_2_Density,HAT_3_5u,HAT_45u,HAT_D50,Filling_Concentration,Filling_Ratio,Batch_Circulation_Hrs,Feed_Hydrate_D50,Feed_Hydrate_SSA,Feed_Hydrate_200,Feed_Hydrate_100,Feed_Hydrate_325,Date
0,2023-01-01 00:00:21,2023-01-01 01:00:00,92.69,198.33,78.54,76.76,53.23,58.68,120.09,1.49,NaN,NaN,NaN,NaN,NaN,71.00,50.64,412.86,62.30,62.98,100.40,165.40,216.89,1.52,1.62,NaN,NaN,NaN,227.2,0.671,NaN,NaN,NaN,80.0,6.0,95.0,2023-01-01
1,2023-01-01 01:00:00,2023-01-01 02:00:00,92.90,194.91,78.96,76.90,53.16,50.27,103.75,1.48,NaN,NaN,NaN,NaN,NaN,71.49,51.85,419.31,62.16,62.89,114.08,170.75,222.75,1.58,1.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,6.0,95.0,2023-01-01
2,2023-01-01 02:00:00,2023-01-01 03:00:00,93.92,195.96,79.59,77.10,52.79,48.76,100.09,1.49,NaN,NaN,NaN,NaN,NaN,66.26,53.48,419.38,63.25,62.82,92.75,144.19,187.43,1.57,1.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,6.0,95.0,2023-01-01
3,2023-01-01 03:00:00,2023-01-01 04:00:00,93.85,190.95,79.78,77.31,52.68,48.69,99.92,1.49,NaN,NaN,NaN,NaN,NaN,71.96,52.76,421.48,63.04,62.75,77.88,149.98,219.66,1.52,1.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,6.0,95.0,2023-01-01
4,2023-01-01 04:00:00,2023-01-01 05:00:00,93.83,193.25,79.76,77.43,52.67,50.88,99.98,1.50,NaN,NaN,NaN,NaN,NaN,69.10,48.16,422.53,62.95,62.72,80.24,166.52,219.01,1.53,1.61,NaN,NaN,NaN,231.5,0.674,NaN,NaN,NaN,80.0,6.0,95.0,2023-01-01


In [625]:
final_Agg_data=Agg_data[final_cols]

In [626]:
treated_agg_data = replace_outliers_with_median(final_Agg_data)

In [627]:
df=initial_data_preprocessing(treated_agg_data)

2023-01-01 01:00:00


In [628]:
resampled_df=data_aggregation(df)

In [629]:
resampled_df.tail()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Feed_Hydrate_200,Feed_Hydrate_325,Feed_Hydrate_100
397,2024-02-02 06:00:00,180.88,230.15,207.865,206.4825,11.248720,126.533707,0.054478,60.49,83.34,68.230,71.231667,6.449222,41.592467,0.090539,114.52,140.38,120.055,124.709583,8.243349,67.952795,0.066100,1.49,1.54,1.52,1.524167,0.011389,0.000130,0.007472,0.173333,0.733333,42.766667,48.332500,71.85,76.03,74.305,74.457917,0.973246,0.947209,0.013071,62.54,373.70,150.235,166.393333,73.988582,5474.310310,0.444661,63.81,142.77,123.145,118.200833,20.502835,420.366225,0.173458,140.18,270.58,190.600,196.332500,22.402562,501.874767,0.114105,0.500000,25.333333,63.333333,0.668571,101.366667,72.0,93.916667,3.0
398,2024-02-03 06:00:00,171.09,211.47,201.900,198.0375,11.846152,140.331324,0.059818,78.45,106.11,92.985,93.191250,7.543664,56.906864,0.080948,132.39,162.69,140.075,143.047500,6.633526,44.003663,0.046373,1.53,1.58,1.57,1.564583,0.015598,0.000243,0.009969,0.136333,0.733333,41.733333,49.533333,74.19,78.16,76.350,76.589167,0.947624,0.897991,0.012373,90.15,353.53,199.960,212.714583,72.559532,5264.885643,0.341112,95.41,155.46,127.085,125.007917,14.182512,201.143661,0.113453,172.77,239.47,205.785,207.934583,13.588103,184.636530,0.065348,0.566667,25.833333,60.166667,0.668333,111.666667,72.0,93.083333,3.0
399,2024-02-04 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.160000,0.700000,37.333333,51.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,18.466667,68.333333,0.673333,103.633333,72.0,93.000000,3.0
400,2024-02-05 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.160000,0.700000,36.150000,51.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450000,20.350000,64.650000,0.678333,100.800000,NaN,NaN,NaN
401,2024-02-06 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.145000,0.600000,27.650000,56.450000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,15.550000,69.250000,0.687500,93.850000,NaN,NaN,NaN


In [630]:
filtered_df=data_filter(resampled_df)

In [631]:
filtered_df=filtered_df[filtered_df['Timestamp_To']<="2023-07-04 22:00:00"]

In [632]:
filtered_df.tail(10)

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Feed_Hydrate_200,Feed_Hydrate_325,Feed_Hydrate_100
175,2023-06-25 06:00:00,103.82,241.07,238.610,226.144167,30.155084,909.329095,0.133345,3.96,79.38,63.270,63.010417,15.394288,236.984109,0.244313,87.21,124.96,119.870,117.620833,7.285766,53.082382,0.061943,1.30,1.56,1.510,1.507917,0.054691,0.002991,0.036269,0.098000,1.333333,40.700000,50.4325,62.53,77.63,74.000,73.630417,3.900960,15.217491,0.052980,0.00,259.92,80.405,97.168333,79.464265,6314.569449,0.817800,85.80,180.16,125.920,128.907917,23.411701,548.107748,0.181616,130.06,235.44,209.56,203.978750,29.170523,850.919429,0.143008,0.633333,6.533333,86.200000,0.663714,91.100000,82.000000,98.000000,2.000000
176,2023-06-26 06:00:00,211.03,237.21,236.100,234.995000,5.167526,26.703322,0.021990,53.87,81.92,70.720,69.012083,8.239188,67.884217,0.119388,113.87,127.93,125.570,124.462500,3.607386,13.013237,0.028984,1.47,1.57,1.520,1.518333,0.027452,0.000754,0.018080,0.108333,1.466667,47.966667,47.6325,63.19,78.86,74.000,73.712083,3.776867,14.264722,0.051238,0.00,299.13,98.745,105.770833,88.916357,7906.118504,0.840651,89.84,178.30,135.455,133.444167,24.558560,603.122878,0.184036,142.60,253.01,201.72,211.635833,25.753329,663.233964,0.121687,0.800000,12.733333,79.466667,0.663143,91.366667,81.083333,98.000000,2.000000
177,2023-06-27 06:00:00,49.17,238.85,187.755,181.828542,58.395049,3409.981721,0.321154,-2.90,102.87,50.050,34.769167,35.094938,1231.654643,1.009370,87.30,120.73,116.890,116.660833,6.802602,46.275391,0.058311,1.48,1.53,1.480,1.493333,0.018572,0.000345,0.012437,0.088000,1.200000,35.300000,54.2150,70.58,79.59,74.000,75.325417,2.294166,5.263200,0.030457,0.01,279.04,26.390,77.874091,97.713259,9547.881035,1.254760,96.82,149.44,120.600,118.094583,10.375154,107.643826,0.087855,172.43,239.93,199.85,202.813333,16.515355,272.756962,0.081431,0.800000,11.700000,82.600000,0.658250,91.700000,80.083333,97.083333,2.000000
178,2023-06-28 06:00:00,205.36,231.86,223.420,223.401250,6.517255,42.474611,0.029173,45.99,82.27,61.765,62.143333,12.695241,161.169136,0.204290,112.21,125.29,121.705,121.891667,2.682197,7.194180,0.022005,1.43,1.57,1.500,1.496667,0.049049,0.002406,0.032772,0.101000,1.300000,46.666667,47.3000,66.17,81.31,74.190,74.180417,3.524026,12.418761,0.047506,0.04,244.52,72.860,76.335000,72.933169,5319.247209,0.955436,82.11,154.32,134.100,132.217500,15.303908,234.209593,0.115748,144.39,259.75,236.39,225.760417,28.946691,837.910917,0.128219,0.833333,14.100000,79.066667,0.659286,95.200000,82.750000,97.916667,2.916667
179,2023-06-29 06:00:00,46.46,23

In [633]:
print(resampled_df.shape)
print(filtered_df.shape)

(402, 69)
(185, 69)


In [634]:
filtered_df = filtered_df.interpolate(method='linear',
                                               limit_direction='forward')
filtered_df = filtered_df.interpolate(method='linear',
                                               limit_direction='backward')

In [635]:
final_df=filtered_df.copy()

In [636]:
max_timestamp = final_df['Timestamp_To'].max()
max_timestamp_dt = pd.to_datetime(max_timestamp)
future_timestamp = max_timestamp_dt + timedelta(days=15)
target_date_str = future_timestamp.strftime("%Y-%m-%d %H:%M")

In [637]:
print(future_timestamp)

2023-07-19 06:00:00


In [638]:
min_days = 39
max_days = 83
shifts_per_day = 1


In [639]:
start_date_min_lag, start_date_max_lag = calculate_lag_dates(target_date_str, min_days, max_days, shifts_per_day)

In [640]:
print(start_date_min_lag, start_date_max_lag)

2023-06-10 06:00:00 2023-04-27 06:00:00


In [641]:
print("Start Date for Minimum Lag:", start_date_min_lag.strftime("%Y-%m-%d %H:%M"))
print("Start Date for Maximum Lag:", start_date_max_lag.strftime("%Y-%m-%d %H:%M"))

Start Date for Minimum Lag: 2023-06-10 06:00
Start Date for Maximum Lag: 2023-04-27 06:00


In [642]:
final_df_lag = final_df[(final_df['Timestamp_To'] <= start_date_min_lag) & (final_df['Timestamp_To'] >= start_date_max_lag)]

In [643]:
final_df_lag.shape

(45, 69)

In [644]:
final_df_lag.head()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Feed_Hydrate_200,Feed_Hydrate_325,Feed_Hydrate_100
116,2023-04-27 06:00:00,150.58,211.40,185.010,182.482083,15.645613,244.785209,0.085738,61.13,85.61,73.095,73.957500,5.937854,35.258107,0.080287,127.24,149.53,142.46,141.649583,4.964740,24.648639,0.035049,1.47,1.54,1.500,1.502500,0.020694,0.000428,0.013773,0.092333,1.133333,39.400000,49.8675,65.19,75.67,74.00,71.841250,3.372000,11.370385,0.046937,0.05,309.00,108.280,109.236250,88.450681,7823.523051,0.809719,78.69,142.85,113.000,114.806667,15.878857,252.138084,0.138310,147.93,250.14,204.430,211.035000,27.945579,780.955365,0.132422,0.833333,18.700000,64.866667,0.659571,82.566667,64.833333,95.916667,3.000000
117,2023-04-28 06:00:00,146.47,217.58,184.670,186.333333,16.771747,281.291493,0.090009,58.58,84.65,71.975,71.714167,7.213420,52.033434,0.100586,131.35,149.24,140.38,141.901250,4.723010,22.306820,0.033284,1.45,1.54,1.495,1.493750,0.026996,0.000729,0.018073,0.087500,1.050000,35.050000,52.1000,61.49,74.50,73.98,72.079167,3.321674,11.033521,0.046084,0.05,292.77,169.475,151.618333,84.988852,7223.104919,0.560545,83.86,145.66,118.315,116.880000,18.633191,347.195791,0.159422,133.27,247.57,194.490,189.167917,23.613368,557.591139,0.124828,0.850000,16.050000,66.150000,0.653286,84.233333,62.250000,95.083333,8.500000
118,2023-04-29 06:00:00,154.75,236.16,169.590,172.390417,16.590208,275.234987,0.096236,60.78,94.40,74.565,75.801250,11.885641,141.268472,0.156800,135.24,157.92,140.32,143.772917,6.413190,41.129004,0.044606,1.46,1.55,1.505,1.502500,0.029525,0.000872,0.019651,0.093333,1.366667,38.500000,50.5000,64.23,75.17,73.58,71.695417,3.460907,11.977878,0.048272,0.14,309.90,130.980,129.228750,92.661895,8586.226716,0.717038,55.63,147.85,106.600,105.823333,20.657125,426.716797,0.195204,173.40,222.57,218.020,211.810833,13.487380,181.909425,0.063677,0.833333,17.166667,66.300000,0.660571,82.200000,63.833333,95.916667,9.000000
119,2023-04-30 06:00:00,163.93,198.48,182.555,181.656667,8.563622,73.335614,0.047142,63.94,84.61,75.225,74.827500,7.956091,63.299385,0.106326,128.52,144.24,139.97,139.042917,3.549269,12.597309,0.025526,1.48,1.54,1.510,1.510417,0.025619,0.000656,0.016962,0.088000,1.166667,34.333333,53.2675,66.53,76.24,74.00,73.110000,2.585394,6.684261,0.035363,1.61,295.03,145.080,141.933333,93.459417,8734.662597,0.658474,78.74,144.93,106.005,105.199583,14.386930,206.983752,0.136758,170.61,249.96,218.815,214.439167,16.393280,268.739625,0.076447,0.800000,14.566667,68.033333,0.652143,81.900000,64.916667,96.000000,8.083333
120,2023-05-01 06:00:00,116.

In [645]:
df=final_df_lag.copy()
df.set_index('Timestamp_To', inplace=True)

In [646]:
for column in df.columns:
    for i in range(21, 72):
        index_to_access = -(i - 20)

        # Ensure the index is within the valid range
        if abs(index_to_access) >= len(df):
            continue  # Skip if the index is out of bounds

        nth_last_value = df[column].iloc[index_to_access]
        new_column_name = f'{column}_shift_{i}'

        df[new_column_name] = pd.NA
        df.at[df.index[-1], new_column_name] = nth_last_value


In [647]:
final_df_lag.head()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Feed_Hydrate_200,Feed_Hydrate_325,Feed_Hydrate_100
116,2023-04-27 06:00:00,150.58,211.40,185.010,182.482083,15.645613,244.785209,0.085738,61.13,85.61,73.095,73.957500,5.937854,35.258107,0.080287,127.24,149.53,142.46,141.649583,4.964740,24.648639,0.035049,1.47,1.54,1.500,1.502500,0.020694,0.000428,0.013773,0.092333,1.133333,39.400000,49.8675,65.19,75.67,74.00,71.841250,3.372000,11.370385,0.046937,0.05,309.00,108.280,109.236250,88.450681,7823.523051,0.809719,78.69,142.85,113.000,114.806667,15.878857,252.138084,0.138310,147.93,250.14,204.430,211.035000,27.945579,780.955365,0.132422,0.833333,18.700000,64.866667,0.659571,82.566667,64.833333,95.916667,3.000000
117,2023-04-28 06:00:00,146.47,217.58,184.670,186.333333,16.771747,281.291493,0.090009,58.58,84.65,71.975,71.714167,7.213420,52.033434,0.100586,131.35,149.24,140.38,141.901250,4.723010,22.306820,0.033284,1.45,1.54,1.495,1.493750,0.026996,0.000729,0.018073,0.087500,1.050000,35.050000,52.1000,61.49,74.50,73.98,72.079167,3.321674,11.033521,0.046084,0.05,292.77,169.475,151.618333,84.988852,7223.104919,0.560545,83.86,145.66,118.315,116.880000,18.633191,347.195791,0.159422,133.27,247.57,194.490,189.167917,23.613368,557.591139,0.124828,0.850000,16.050000,66.150000,0.653286,84.233333,62.250000,95.083333,8.500000
118,2023-04-29 06:00:00,154.75,236.16,169.590,172.390417,16.590208,275.234987,0.096236,60.78,94.40,74.565,75.801250,11.885641,141.268472,0.156800,135.24,157.92,140.32,143.772917,6.413190,41.129004,0.044606,1.46,1.55,1.505,1.502500,0.029525,0.000872,0.019651,0.093333,1.366667,38.500000,50.5000,64.23,75.17,73.58,71.695417,3.460907,11.977878,0.048272,0.14,309.90,130.980,129.228750,92.661895,8586.226716,0.717038,55.63,147.85,106.600,105.823333,20.657125,426.716797,0.195204,173.40,222.57,218.020,211.810833,13.487380,181.909425,0.063677,0.833333,17.166667,66.300000,0.660571,82.200000,63.833333,95.916667,9.000000
119,2023-04-30 06:00:00,163.93,198.48,182.555,181.656667,8.563622,73.335614,0.047142,63.94,84.61,75.225,74.827500,7.956091,63.299385,0.106326,128.52,144.24,139.97,139.042917,3.549269,12.597309,0.025526,1.48,1.54,1.510,1.510417,0.025619,0.000656,0.016962,0.088000,1.166667,34.333333,53.2675,66.53,76.24,74.00,73.110000,2.585394,6.684261,0.035363,1.61,295.03,145.080,141.933333,93.459417,8734.662597,0.658474,78.74,144.93,106.005,105.199583,14.386930,206.983752,0.136758,170.61,249.96,218.815,214.439167,16.393280,268.739625,0.076447,0.800000,14.566667,68.033333,0.652143,81.900000,64.916667,96.000000,8.083333
120,2023-05-01 06:00:00,116.

In [648]:
final_df_lag.tail()

,Timestamp_To,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Feed_Hydrate_200,Feed_Hydrate_325,Feed_Hydrate_100
156,2023-06-06 06:00:00,114.08,152.34,135.400,136.864000,9.001715,81.030878,0.065771,68.63,82.57,78.470,77.988000,3.151856,9.934196,0.040415,138.25,152.55,149.160,148.794737,3.079119,9.480971,0.020694,1.48,1.51,1.51,1.503684,0.008951,0.000080,0.005953,0.087667,1.033333,41.400000,49.6000,73.65,77.83,76.440,76.289000,1.017152,1.034599,0.013333,35.29,268.77,172.205,164.793000,67.831374,4601.095275,0.411616,94.79,153.14,118.550,123.753500,19.523801,381.178813,0.157764,173.92,240.62,202.110,207.569474,19.632831,385.448050,0.094584,0.766667,19.200000,69.666667,0.658571,87.166667,72.272727,95.000000,2.000000
157,2023-06-07 06:00:00,132.61,171.19,160.115,156.849583,9.904968,98.108395,0.063149,69.19,82.54,76.350,76.917083,3.844271,14.778422,0.049979,143.54,154.49,148.160,148.106667,2.532015,6.411101,0.017096,1.48,1.52,1.50,1.501250,0.012619,0.000159,0.008406,0.094000,1.100000,47.766667,47.1250,72.60,84.13,76.415,77.065833,2.636437,6.950799,0.034210,80.83,335.12,166.600,179.511667,65.929486,4346.697101,0.367271,87.54,152.21,121.520,118.818750,17.815894,317.406090,0.149942,151.81,225.72,201.535,203.712917,17.445521,304.346204,0.085638,0.733333,21.000000,66.433333,0.656429,88.033333,72.000000,95.000000,2.000000
158,2023-06-08 06:00:00,130.24,166.54,146.825,147.211667,8.071752,65.153180,0.054831,62.61,88.34,77.805,77.025833,4.920017,24.206564,0.063875,126.08,153.24,145.745,145.477917,5.280621,27.884956,0.036298,1.46,1.54,1.51,1.505417,0.016413,0.000269,0.010903,0.086667,1.066667,38.800000,50.4325,75.21,81.52,77.665,78.035000,2.188786,4.790783,0.028049,63.84,276.81,168.515,174.041667,61.046068,3726.622371,0.350755,100.84,153.57,116.855,121.310417,15.023955,225.719222,0.123847,180.98,225.80,216.095,210.502917,12.230132,149.576135,0.058100,0.700000,16.000000,73.666667,0.657571,87.966667,72.916667,95.916667,2.916667
159,2023-06-09 06:00:00,114.55,138.04,122.500,124.220417,6.769123,45.821030,0.054493,72.88,83.09,80.180,79.569167,2.460882,6.055938,0.030928,128.98,147.21,137.895,138.365833,4.142778,17.162608,0.029941,1.51,1.54,1.53,1.527083,0.008587,0.000074,0.005623,0.091333,1.133333,41.500000,49.6000,76.49,80.63,77.905,78.035000,0.890227,0.792504,0.011408,63.67,281.35,198.200,189.050417,53.114494,2821.149526,0.280954,85.79,154.36,114.940,116.369167,18.078476,326.831304,0.155355,146.31,244.43,198.750,190.672917,27.563855,759.766091,0.144561,0.766667,18.533333,67.933333,0.660286,86.400000,75.750000,96.000000,2.083333
160,2023-06-10 06:00:00,122.60,162

In [649]:
df.head()

,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Feed_Hydrate_200,Feed_Hydrate_325,Feed_Hydrate_100,Special_Filling_Flow_min_shift_21,Special_Filling_Flow_min_shift_22,Special_Filling_Flow_min_shift_23,Special_Filling_Flow_min_shift_24,Special_Filling_Flow_min_shift_25,Special_Filling_Flow_min_shift_26,Special_Filling_Flow_min_shift_27,Special_Filling_Flow_min_shift_28,Special_Filling_Flow_min_shift_29,Special_Filling_Flow_min_shift_30,Special_Filling_Flow_min_shift_31,Special_Filling_Flow_min_shift_32,Special_Filling_Flow_min_shift_33,Special_Filling_Flow_min_shift_34,Special_Filling_Flow_min_shift_35,Special_Filling_Flow_min_shift_36,Special_Filling_Flow_min_shift_37,Special_Filling_Flow_min_shift_38,Special_Filling_Flow_min_shift_39,Special_Filling_Flow_min_shift_40,Special_Filling_Flow_min_shift_41,Special_Filling_Flow_min_shift_42,Special_Filling_Flow_min_shift_43,Special_Filling_Flow_min_shift_44,Special_Filling_Flow_min_shift_45,Special_Filling_Flow_min_shift_46,Special_Filling_Flow_min_shift_47,Special_Filling_Flow_min_shift_48,Special_Filling_Flow_min_shift_49,Special_Filling_Flow_min_shift_50,Special_Filling_Flow_min_shift_51,Special_Filling_Flow_min_shift_52,Special_Filling_Flow_min_shift_53,Special_Filling_Flow_min_shift_54,Special_Filling_Flow_min_shift_55,Special_Filling_Flow_min_shift_56,Special_Filling_Flow_min_shift_57,Special_Filling_Flow_min_shift_58,Special_Filling_Flow_min_shift_59,Special_Filling_Flow_min_shift_60,Special_Filling_Flow_min_shift_61,Special_Filling_Flow_min_shift_62,Special_Filling_Flow_min_shift_63,Special_Filling_Flow_min_shift_64,Special_Filling_Flow_max_shift_21,Special_Filling_Flow_max_shift_22,Special_Filling_Flow_max_shift_23,Special_Filling_Flow_max_shift_24,Special_Filling_Flow_max_shift_25,Special_Filling_Flow_max_shift_26,Special_Filling_Flow_max_shift_27,Special_Filling_Flow_max_shift_28,Special_Filling_Flow_max_shift_29,Special_Filling_Flow_max_shift_30,Special_Filling_Flow_max_shift_31,Special_Filling_Flow_max_shift_32,Special_Filling_Flow_max_shift_33,Special_Filling_Flow_max_shift_34,Special_Filling_Flow_max_shift_35,Special_Filling_Flow_max_shift_36,Special_Filling_Flow_max_shift_37,Special_Filling_Flow_max_shift_38,Special_Filling_Flow_max_shift_39,Special_Filling_Flow_max_shift_40,Special_Filling_Flow_max_shift_41,Special_Filling_Flow_max_shift_42,Special_Filling_Flow_max_shift_43,Special_Filling_Flow_max_shift_44,Special_Filling_Flow_max_shift_45,Special_Filling_Flow_max_shift_46,Special_Filling_Flow_max_shift_47,Special_Filling_Flow_max_shift_48,Special_Filling_Flow_max_shift_49,Specia

In [650]:
df.tail()

,Special_Filling_Flow_min,Special_Filling_Flow_max,Special_Filling_Flow_median,Special_Filling_Flow_mean,Special_Filling_Flow_std,Special_Filling_Flow_var,Special_Filling_Flow_var_coeff,Fine_Seed_Tonnage_min,Fine_Seed_Tonnage_max,Fine_Seed_Tonnage_median,Fine_Seed_Tonnage_mean,Fine_Seed_Tonnage_std,Fine_Seed_Tonnage_var,Fine_Seed_Tonnage_var_coeff,Fine_Seed_Flow_min,Fine_Seed_Flow_max,Fine_Seed_Flow_median,Fine_Seed_Flow_mean,Fine_Seed_Flow_std,Fine_Seed_Flow_var,Fine_Seed_Flow_var_coeff,Fine_Seed_Density_min,Fine_Seed_Density_max,Fine_Seed_Density_median,Fine_Seed_Density_mean,Fine_Seed_Density_std,Fine_Seed_Density_var,Fine_Seed_Density_var_coeff,Fine_Seed_SSA,Fine_Seed_3_5u,Fine_Seed_45u,Fine_Seed_D50,PHE_Inlet_Temp_min,PHE_Inlet_Temp_max,PHE_Inlet_Temp_median,PHE_Inlet_Temp_mean,PHE_Inlet_Temp_std,PHE_Inlet_Temp_var,PHE_Inlet_Temp_var_coeff,Regular_Filling_Flow_min,Regular_Filling_Flow_max,Regular_Filling_Flow_median,Regular_Filling_Flow_mean,Regular_Filling_Flow_std,Regular_Filling_Flow_var,Regular_Filling_Flow_var_coeff,Coarse_Seed_1_Tonnage_min,Coarse_Seed_1_Tonnage_max,Coarse_Seed_1_Tonnage_median,Coarse_Seed_1_Tonnage_mean,Coarse_Seed_1_Tonnage_std,Coarse_Seed_1_Tonnage_var,Coarse_Seed_1_Tonnage_var_coeff,Coarse_Seed_2_Flow_min,Coarse_Seed_2_Flow_max,Coarse_Seed_2_Flow_median,Coarse_Seed_2_Flow_mean,Coarse_Seed_2_Flow_std,Coarse_Seed_2_Flow_var,Coarse_Seed_2_Flow_var_coeff,HAT_3_5u,HAT_45u,HAT_D50,Filling_Ratio,Feed_Hydrate_D50,Feed_Hydrate_200,Feed_Hydrate_325,Feed_Hydrate_100,Special_Filling_Flow_min_shift_21,Special_Filling_Flow_min_shift_22,Special_Filling_Flow_min_shift_23,Special_Filling_Flow_min_shift_24,Special_Filling_Flow_min_shift_25,Special_Filling_Flow_min_shift_26,Special_Filling_Flow_min_shift_27,Special_Filling_Flow_min_shift_28,Special_Filling_Flow_min_shift_29,Special_Filling_Flow_min_shift_30,Special_Filling_Flow_min_shift_31,Special_Filling_Flow_min_shift_32,Special_Filling_Flow_min_shift_33,Special_Filling_Flow_min_shift_34,Special_Filling_Flow_min_shift_35,Special_Filling_Flow_min_shift_36,Special_Filling_Flow_min_shift_37,Special_Filling_Flow_min_shift_38,Special_Filling_Flow_min_shift_39,Special_Filling_Flow_min_shift_40,Special_Filling_Flow_min_shift_41,Special_Filling_Flow_min_shift_42,Special_Filling_Flow_min_shift_43,Special_Filling_Flow_min_shift_44,Special_Filling_Flow_min_shift_45,Special_Filling_Flow_min_shift_46,Special_Filling_Flow_min_shift_47,Special_Filling_Flow_min_shift_48,Special_Filling_Flow_min_shift_49,Special_Filling_Flow_min_shift_50,Special_Filling_Flow_min_shift_51,Special_Filling_Flow_min_shift_52,Special_Filling_Flow_min_shift_53,Special_Filling_Flow_min_shift_54,Special_Filling_Flow_min_shift_55,Special_Filling_Flow_min_shift_56,Special_Filling_Flow_min_shift_57,Special_Filling_Flow_min_shift_58,Special_Filling_Flow_min_shift_59,Special_Filling_Flow_min_shift_60,Special_Filling_Flow_min_shift_61,Special_Filling_Flow_min_shift_62,Special_Filling_Flow_min_shift_63,Special_Filling_Flow_min_shift_64,Special_Filling_Flow_max_shift_21,Special_Filling_Flow_max_shift_22,Special_Filling_Flow_max_shift_23,Special_Filling_Flow_max_shift_24,Special_Filling_Flow_max_shift_25,Special_Filling_Flow_max_shift_26,Special_Filling_Flow_max_shift_27,Special_Filling_Flow_max_shift_28,Special_Filling_Flow_max_shift_29,Special_Filling_Flow_max_shift_30,Special_Filling_Flow_max_shift_31,Special_Filling_Flow_max_shift_32,Special_Filling_Flow_max_shift_33,Special_Filling_Flow_max_shift_34,Special_Filling_Flow_max_shift_35,Special_Filling_Flow_max_shift_36,Special_Filling_Flow_max_shift_37,Special_Filling_Flow_max_shift_38,Special_Filling_Flow_max_shift_39,Special_Filling_Flow_max_shift_40,Special_Filling_Flow_max_shift_41,Special_Filling_Flow_max_shift_42,Special_Filling_Flow_max_shift_43,Special_Filling_Flow_max_shift_44,Special_Filling_Flow_max_shift_45,Special_Filling_Flow_max_shift_46,Special_Filling_Flow_max_shift_47,Special_Filling_Flow_max_shift_48,Special_Filling_Flow_max_shift_49,Specia

In [651]:
df_lag_cleaned = df.dropna()

In [652]:
df_lag_cleaned.shape

(1, 3060)

In [653]:
best_cols = 'D:/BGM_Codebase/+200mesh_test/model_pickles/precip_final_cols.pkl'
with open(best_cols, 'rb') as file:
    final_cols = pickle.load(file)

In [654]:
final_cols

['Feed_Hydrate_100',
 'Feed_Hydrate_200',
 'Feed_Hydrate_325',
 'Fine_Seed_Flow_median_shift_38',
 'Fine_Seed_Tonnage_max_shift_43',
 'Fine_Seed_Tonnage_mean_shift_42',
 'HAT_45u_shift_26',
 'HAT_45u_shift_32',
 'HAT_45u_shift_33',
 'HAT_45u_shift_41',
 'HAT_45u_shift_42',
 'HAT_45u_shift_49',
 'HAT_D50_shift_21',
 'HAT_D50_shift_22',
 'HAT_D50_shift_25',
 'HAT_D50_shift_26',
 'HAT_D50_shift_29',
 'HAT_D50_shift_31',
 'Regular_Filling_Flow_max_shift_37']

In [655]:
filtered_cols = [col for col in final_cols if col not in ["Feed_Hydrate_100", "Feed_Hydrate_200", "Feed_Hydrate_325"] and int(col.split('_')[-1]) >= 21]

# Add the target columns
filtered_cols.extend(["Feed_Hydrate_100", "Feed_Hydrate_200", "Feed_Hydrate_325"])


In [656]:
filtered_cols

['Fine_Seed_Flow_median_shift_38',
 'Fine_Seed_Tonnage_max_shift_43',
 'Fine_Seed_Tonnage_mean_shift_42',
 'HAT_45u_shift_26',
 'HAT_45u_shift_32',
 'HAT_45u_shift_33',
 'HAT_45u_shift_41',
 'HAT_45u_shift_42',
 'HAT_45u_shift_49',
 'HAT_D50_shift_21',
 'HAT_D50_shift_22',
 'HAT_D50_shift_25',
 'HAT_D50_shift_26',
 'HAT_D50_shift_29',
 'HAT_D50_shift_31',
 'Regular_Filling_Flow_max_shift_37',
 'Feed_Hydrate_100',
 'Feed_Hydrate_200',
 'Feed_Hydrate_325']

In [657]:
df_final_cleaned=df_lag_cleaned[filtered_cols]

In [658]:
df_final_cleaned.shape

(1, 19)

In [659]:
print (df_final_cleaned)

                    Fine_Seed_Flow_median_shift_38 Fine_Seed_Tonnage_max_shift_43 Fine_Seed_Tonnage_mean_shift_42 HAT_45u_shift_26 HAT_45u_shift_32 HAT_45u_shift_33 HAT_45u_shift_41 HAT_45u_shift_42 HAT_45u_shift_49 HAT_D50_shift_21 HAT_D50_shift_22 HAT_D50_shift_25 HAT_D50_shift_26 HAT_D50_shift_29 HAT_D50_shift_31 Regular_Filling_Flow_max_shift_37  Feed_Hydrate_100  Feed_Hydrate_200  Feed_Hydrate_325
Timestamp_To                                                                                                                                                                                                                                                                                                                                                                                                         
2023-06-10 06:00:00                        149.115                          85.61                       66.025217             21.1        22.016667        22.583333            26.55       

In [660]:
import pandas as pd
import pickle
import numpy as np

# Define the multi-target columns
target_columns = ['Feed_Hydrate_100', 'Feed_Hydrate_200', 'Feed_Hydrate_325']

# Prepare input features (X)
X_unseen = df_final_cleaned.drop(columns=target_columns).copy()
y_unseen = df_final_cleaned[target_columns].copy()

# Convert object columns to numeric
for col in X_unseen.columns:
    if X_unseen[col].dtype == 'object':
        X_unseen[col] = pd.to_numeric(X_unseen[col], errors='coerce')

# Handle missing values
X_unseen.fillna(method='ffill', inplace=True)

# Load the multi-output model
model_path = 'D:/BGM_Codebase/+200mesh_test/model_pickles_22-01-2024/Light_gbm_model_22-01-2024.pkl'
with open(model_path, 'rb') as file:
    model = pickle.load(file)

# Ensure X_unseen has at least 2D shape
if len(X_unseen.shape) == 1:
    X_unseen = X_unseen.values.reshape(1, -1)

# Run prediction
y_pred = model.predict(X_unseen)

# Handle unexpected 1D output
if isinstance(y_pred, np.ndarray) and y_pred.ndim == 1:
    if len(target_columns) == 1:
        y_pred_df = pd.DataFrame(y_pred, columns=target_columns)
    elif len(y_pred) == len(target_columns):  # single row, multi-output
        y_pred_df = pd.DataFrame([y_pred], columns=target_columns)
    else:
        raise ValueError(f"Prediction shape mismatch. Got shape {y_pred.shape}, expected {len(target_columns)} outputs.")
else:
    y_pred_df = pd.DataFrame(y_pred, columns=target_columns)

# Show predictions
print(y_pred_df.head())


ValueError: Prediction shape mismatch. Got shape (1,), expected 3 outputs.